In [1]:
#topic modeling option 1: https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0


In [2]:
import pandas as pd
from pathlib import Path
import re

#PREPARE DATA FOR LDA ANALYSIS
import gensim #had AttributeError: module 'smart_open' has not attribute 'local_file'
## conda install -c anaconda --update-deps gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) \
             if word not in stop_words] for doc in texts]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\F0040RP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
""" textdir = Path("~/shared/RR-workshop-data/state-of-the-union-dataset/txt").expanduser() 
pathlist = textdir.glob("*.txt")
textlists = []
for path in pathlist:
    #print(path.stem)
    pres, year = path.stem.split("_")
    #print(year," - ",pres)
    
    with open(path,encoding = 'utf-8') as f:
        txt = f.read()
    if year == "1790":
        print(txt[:400])
    textlists.append([year,pres,txt])

textdf = pd.DataFrame(textlists,columns = ["year","president","text"])
textdf.head() """

' textdir = Path("~/shared/RR-workshop-data/state-of-the-union-dataset/txt").expanduser() \npathlist = textdir.glob("*.txt")\ntextlists = []\nfor path in pathlist:\n    #print(path.stem)\n    pres, year = path.stem.split("_")\n    #print(year," - ",pres)\n    \n    with open(path,encoding = \'utf-8\') as f:\n        txt = f.read()\n    if year == "1790":\n        print(txt[:400])\n    textlists.append([year,pres,txt])\n\ntextdf = pd.DataFrame(textlists,columns = ["year","president","text"])\ntextdf.head() '

In [4]:
textdir = Path("~/shared/RR-workshop-data/text_datasets").expanduser()
textdf = pd.read_csv("nyt_df_2023-10.csv", encoding = 'utf-8')
textdf.head()

,Unnamed: 0,abstract,web_url,snippet,lead_paragraph,source,multimedia,keywords,pub_date,document_type,...,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,print_section,print_page
0,0,The new chairman of the Senate Foreign Relatio...,https://www.nytimes.com/2023/09/30/us/politics...,The new chairman of the Senate Foreign Relatio...,"Senator Ben Cardin of Maryland, the new chairm...",The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'glocations', 'value': 'Egypt', 'ran...",2023-10-01T00:06:00+0000,article,...,NaN,NaN,NaN,NaN,NaN,By Karoun Demirjian,"[{'firstname': 'Karoun', 'middlename': None, '...",NaN,NaN,NaN
1,1,The Senate overwhelmingly passed a bill to fun...,https://www.nytimes.com/interactive/2023/09/30...,The Senate overwhelmingly passed a bill to fun...,The Senate overwhelmingly passed a bill to fun...,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Shutdowns (Inst...",2023-10-01T00:06:25+0000,multimedia,...,NaN,NaN,NaN,NaN,NaN,"By Lazaro Gamio, Alicia Parlapiano, Jasmine C....","[{'firstname': 'Lazaro', 'middlename': None, '...",NaN,NaN,NaN
2,2,"In much of Europe, the election in Slovakia wa...",https://www.nytimes.com/2023/09/30/world/europ...,"In much of Europe, the election in Slovakia wa...",A Russia-friendly populist party finished firs...,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'glocations', 'value': 'Slovakia', '...",2023-10-01T00:18:47+0000,article,...,NaN,NaN,NaN,NaN,NaN,By Andrew Higgins,"[{'firstname': 'Andrew', 'middlename': None, '...",NaN,NaN,NaN
3,3,"Gideon Cody, who orchestrated a widely critici...",https://www.nytimes.com/2023/09/30/us/marion-k...,"Gideon Cody, who orchestrated a widely critici...","The police chief of Marion, Kansas, was suspen...",The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Newspapers', 'r...",2023-10-01T00:47:45+0000,article,...,NaN,Police Chief Who Ordered Newspaper Raid Is Sus...,NaN,NaN,NaN,By Orlando Mayorquin,"[{'firstname': 'Orlando', 'middlename': None, ...",NaN,A,21.0
4,4,The assistant general manager of the Calgary F...,https://www.nytimes.com/2023/09/30/sports/chri...,The assistant general manager of the Calgary F...,"Chris Snow, the assistant general manager of t...",The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'persons', 'value': 'Snow, Chris', '...",2023-10-01T01:38:51+0000,article,...,NaN,"Chris Snow, 42, Hockey Executive Who ‘Never St...",NaN,NaN,NaN,By Sopan Deb,"[{'firstname': 'Sopan', 'middlename': None, 'l...",NaN,B,10.0


In [5]:
#https://stackoverflow.com/questions/39291499/how-to-concatenate-multiple-column-values-into-a-single-column-in-pandas-datafra
textdf['text'] = textdf[['abstract', 'lead_paragraph', 'headline.main']].apply(lambda row: '\n'.join(row.values.astype(str)), axis = 1)
textdf.head()

,Unnamed: 0,abstract,web_url,snippet,lead_paragraph,source,multimedia,keywords,pub_date,document_type,...,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,print_section,print_page,text
0,0,The new chairman of the Senate Foreign Relatio...,https://www.nytimes.com/2023/09/30/us/politics...,The new chairman of the Senate Foreign Relatio...,"Senator Ben Cardin of Maryland, the new chairm...",The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'glocations', 'value': 'Egypt', 'ran...",2023-10-01T00:06:00+0000,article,...,NaN,NaN,NaN,NaN,By Karoun Demirjian,"[{'firstname': 'Karoun', 'middlename': None, '...",NaN,NaN,NaN,The new chairman of the Senate Foreign Relatio...
1,1,The Senate overwhelmingly passed a bill to fun...,https://www.nytimes.com/interactive/2023/09/30...,The Senate overwhelmingly passed a bill to fun...,The Senate overwhelmingly passed a bill to fun...,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Shutdowns (Inst...",2023-10-01T00:06:25+0000,multimedia,...,NaN,NaN,NaN,NaN,"By Lazaro Gamio, Alicia Parlapiano, Jasmine C....","[{'firstname': 'Lazaro', 'middlename': None, '...",NaN,NaN,NaN,The Senate overwhelmingly passed a bill to fun...
2,2,"In much of Europe, the election in Slovakia wa...",https://www.nytimes.com/2023/09/30/world/europ...,"In much of Europe, the election in Slovakia wa...",A Russia-friendly populist party finished firs...,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'glocations', 'value': 'Slovakia', '...",2023-10-01T00:18:47+0000,article,...,NaN,NaN,NaN,NaN,By Andrew Higgins,"[{'firstname': 'Andrew', 'middlename': None, '...",NaN,NaN,NaN,"In much of Europe, the election in Slovakia wa..."
3,3,"Gideon Cody, who orchestrated a widely critici...",https://www.nytimes.com/2023/09/30/us/marion-k...,"Gideon Cody, who orchestrated a widely critici...","The police chief of Marion, Kansas, was suspen...",The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Newspapers', 'r...",2023-10-01T00:47:45+0000,article,...,Police Chief Who Ordered Newspaper Raid Is Sus...,NaN,NaN,NaN,By Orlando Mayorquin,"[{'firstname': 'Orlando', 'middlename': None, ...",NaN,A,21.0,"Gideon Cody, who orchestrated a widely critici..."
4,4,The assistant general manager of the Calgary F...,https://www.nytimes.com/2023/09/30/sports/chri...,The assistant general manager of the Calgary F...,"Chris Snow, the assistant general manager of t...",The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'persons', 'value': 'Snow, Chris', '...",2023-10-01T01:38:51+0000,article,...,"Chris Snow, 42, Hockey Executive Who ‘Never St...",NaN,NaN,NaN,By Sopan Deb,"[{'firstname': 'Sopan', 'middlename': None, 'l...",NaN,B,10.0,The assistant general manager of the Calgary F...


In [6]:
textdf.loc[0, 'text']

'The new chairman of the Senate Foreign Relations Committee could force the Biden administration to reverse its decision to send Egypt $235 million in military assistance.\nSenator Ben Cardin of Maryland, the new chairman of the Senate Foreign Relations Committee, pledged on Saturday to block the release of $235 million in military aid to Egypt, a move that could force the Biden administration to reverse its decision to prioritize national security interests over Congress’s concerns about the country’s human rights record.\nSenate Democrat Threatens to Block More of Military Aid to Egypt'

In [7]:
textdf['text_processed'] = textdf['text'].map(lambda x: re.sub("[,\.!?]", '', x))
textdf['text_processed'] = textdf['text'].map(lambda x: x.lower())
#textdf = textdf.sort_values(by = "year")
textdf.head()

,Unnamed: 0,abstract,web_url,snippet,lead_paragraph,source,multimedia,keywords,pub_date,document_type,...,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,print_section,print_page,text,text_processed
0,0,The new chairman of the Senate Foreign Relatio...,https://www.nytimes.com/2023/09/30/us/politics...,The new chairman of the Senate Foreign Relatio...,"Senator Ben Cardin of Maryland, the new chairm...",The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'glocations', 'value': 'Egypt', 'ran...",2023-10-01T00:06:00+0000,article,...,NaN,NaN,NaN,By Karoun Demirjian,"[{'firstname': 'Karoun', 'middlename': None, '...",NaN,NaN,NaN,The new chairman of the Senate Foreign Relatio...,the new chairman of the senate foreign relatio...
1,1,The Senate overwhelmingly passed a bill to fun...,https://www.nytimes.com/interactive/2023/09/30...,The Senate overwhelmingly passed a bill to fun...,The Senate overwhelmingly passed a bill to fun...,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Shutdowns (Inst...",2023-10-01T00:06:25+0000,multimedia,...,NaN,NaN,NaN,"By Lazaro Gamio, Alicia Parlapiano, Jasmine C....","[{'firstname': 'Lazaro', 'middlename': None, '...",NaN,NaN,NaN,The Senate overwhelmingly passed a bill to fun...,the senate overwhelmingly passed a bill to fun...
2,2,"In much of Europe, the election in Slovakia wa...",https://www.nytimes.com/2023/09/30/world/europ...,"In much of Europe, the election in Slovakia wa...",A Russia-friendly populist party finished firs...,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'glocations', 'value': 'Slovakia', '...",2023-10-01T00:18:47+0000,article,...,NaN,NaN,NaN,By Andrew Higgins,"[{'firstname': 'Andrew', 'middlename': None, '...",NaN,NaN,NaN,"In much of Europe, the election in Slovakia wa...","in much of europe, the election in slovakia wa..."
3,3,"Gideon Cody, who orchestrated a widely critici...",https://www.nytimes.com/2023/09/30/us/marion-k...,"Gideon Cody, who orchestrated a widely critici...","The police chief of Marion, Kansas, was suspen...",The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Newspapers', 'r...",2023-10-01T00:47:45+0000,article,...,NaN,NaN,NaN,By Orlando Mayorquin,"[{'firstname': 'Orlando', 'middlename': None, ...",NaN,A,21.0,"Gideon Cody, who orchestrated a widely critici...","gideon cody, who orchestrated a widely critici..."
4,4,The assistant general manager of the Calgary F...,https://www.nytimes.com/2023/09/30/sports/chri...,The assistant general manager of the Calgary F...,"Chris Snow, the assistant general manager of t...",The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'persons', 'value': 'Snow, Chris', '...",2023-10-01T01:38:51+0000,article,...,NaN,NaN,NaN,By Sopan Deb,"[{'firstname': 'Sopan', 'middlename': None, 'l...",NaN,B,10.0,The assistant general manager of the Calgary F...,the assistant general manager of the calgary f...


In [8]:
#combined texts
#textdf2 = pd.read_csv(Path(textdir.parent, "sotu_bypres.tsv"), sep="\t", encoding = 'utf-8')
#textdf2.head()

In [9]:
text_list = textdf['text_processed'].values.tolist()
word_list = list(sent_to_words(text_list))
word_list = remove_stopwords(word_list)

In [10]:
import gensim.corpora as corpora 
id2word = corpora.Dictionary(word_list) # create dictionary
texts = word_list
corpus = [id2word.doc2bow(text) for text in texts] # Term Document Frequency
print(corpus[:1][0][:30]) # View

[(0, 2), (1, 2), (2, 1), (3, 1), (4, 2), (5, 2), (6, 1), (7, 2), (8, 2), (9, 1), (10, 1), (11, 2), (12, 1), (13, 2), (14, 1), (15, 3), (16, 2), (17, 2), (18, 1), (19, 1), (20, 1), (21, 3), (22, 2), (23, 1), (24, 1), (25, 2), (26, 1), (27, 1), (28, 1), (29, 2)]


In [11]:

#LDA MODEL TRAINING
from pprint import pprint
# number of topics
num_topics = 100  
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(49,
  '0.007*"new" + 0.006*"gaza" + 0.005*"officials" + 0.005*"israel" + '
  '0.004*"two" + 0.004*"york" + 0.004*"could" + 0.004*"said" + 0.004*"war" + '
  '0.004*"people"'),
 (30,
  '0.006*"israel" + 0.005*"would" + 0.004*"said" + 0.004*"two" + 0.004*"week" '
  '+ 0.004*"gaza" + 0.003*"fall" + 0.003*"biden" + 0.003*"last" + '
  '0.003*"american"'),
 (52,
  '0.009*"new" + 0.006*"house" + 0.005*"night" + 0.004*"one" + 0.004*"speaker" '
  '+ 0.003*"said" + 0.003*"southern" + 0.003*"representative" + 0.003*"case" + '
  '0.003*"hospital"'),
 (26,
  '0.005*"israel" + 0.005*"new" + 0.005*"day" + 0.005*"said" + 0.005*"first" + '
  '0.005*"wordle" + 0.004*"years" + 0.004*"gaza" + 0.004*"make" + '
  '0.004*"people"'),
 (77,
  '0.006*"music" + 0.005*"year" + 0.005*"military" + 0.005*"black" + '
  '0.005*"one" + 0.005*"new" + 0.004*"said" + 0.004*"first" + 0.004*"ukraine" '
  '+ 0.004*"aid"'),
 (87,
  '0.011*"gaza" + 0.010*"israel" + 0.006*"people" + 0.006*"new" + '
  '0.004*"killed" + 0.004*"o

In [12]:
#ANALYZING LDA MODEL RESULTS

import pyLDAvis
# import pyLDAvis.gensim
import pyLDAvis.gensim_models
import pickle 

In [13]:
# Visualize the topics
pyLDAvis.enable_notebook()
#LDAvis_data_filepath = os.path.join('./results/sotuByPres_ldavis_prepared_'+str(num_topics))
LDAvis_data_filepath = Path('sotu_ldavis_prepared' + str(num_topics))


In [14]:
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

pyLDAvis.save_html(LDAvis_prepared, 'sotu_ldavis_prepared_'+ str(num_topics) +'.html')

LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
53     0.062081 -0.043711       1        1  1.872018
37     0.016397 -0.009242       2        1  1.704852
64     0.015370  0.000954       3        1  1.677220
74    -0.063251  0.012318       4        1  1.590560
0      0.044246  0.032159       5        1  1.581225
...         ...       ...     ...      ...       ...
22    -0.030792  0.016750      96        1  0.642960
18     0.000178  0.004922      97        1  0.627523
1      0.043379  0.038086      98        1  0.623412
77    -0.040940 -0.003760      99        1  0.608149
9     -0.043457  0.035768     100        1  0.576353

[100 rows x 5 columns], topic_info=          Term        Freq       Total  Category  logprob  loglift
254        bee  109.000000  109.000000   Default  30.0000  30.0000
277   spelling  111.000000  111.000000   Default  29.0000  29.0000
811     israel  923.000000  923.000000   Default  28.0000  28.0000
246      trump  339.000000  339.000000   Default  27.0000  27.0000
9670      gaza  590.000000  590.000000   Default  26.0000  26.0000
...        ...         ...         ...       ...      ...      ...
2006      film    2.220870  106.443545  Topic100  -6.0667   1.2865
1490    killed    2.258493  161.170030  Topic100  -6.0499   0.8884
484     around    2.267711  174.943434  Topic100  -6.0459   0.8105
11       could    2.295364  301.743127  Topic100  -6.0337   0.2775
61       first    2.243932  357.643177  Topic100  -6.0564   0.0849

[8054 rows x 6 columns], token_table=       Topic      Freq           Term
term                                 
13218     13  0.385095        aaliyah
19733      5  0.477420         aarhus
597       74  0.769818          abbot
11108     97  0.604418  abbreviations
8165      31  0.303671          abdel
...      ...       ...            ...
11159     42  0.401417       zoonotic
5807      58  0.599068       zubaydah
8223      41  0.579710          zumba
1694       5  0.477741         zweden
9942      38  0.579845             知音

[39872 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[54, 38, 65, 75, 1, 33, 89, 30, 52, 94, 69, 62, 81, 96, 37, 28, 4, 97, 3, 5, 39, 47, 98, 32, 72, 71, 91, 51, 18, 63, 88, 21, 40, 35, 9, 53, 60, 42, 41, 66, 87, 79, 46, 24, 95, 13, 50, 76, 77, 67, 61, 59, 45, 57, 93, 85, 55, 58, 48, 70, 82, 56, 80, 12, 49, 11, 74, 36, 73, 17, 25, 29, 27, 34, 20, 64, 92, 14, 90, 8, 86, 44, 22, 43, 100, 84, 6, 83, 15, 68, 26, 16, 7, 99, 31, 23, 19, 2, 78, 10])

In [15]:
LDAvis_data_filepath

WindowsPath('sotu_ldavis_prepared100')

In [16]:
num_topics

100